# Nonconvex QP

**Adapted from**:  of [Floudas1999; Section 2.2](@cite), [Lasserre2009; Table 5.1](@cite)

## Introduction

Consider the nonconvex Quadratic Program (QP) from [Floudas1999; Section 2.2](@cite)
that minimizes the *concave* function $c^\top x - x^\top Qx / 2$
over the polyhedron obtained by intersecting the hypercube $[0, 1]^5$
with the halfspace $10x_1 + 12x_2 + 11x_3 + 7x_4 + 4x_5 \le 40$.

In [1]:
using LinearAlgebra
c = [42, 44, 45, 47, 47.5]
Q = 100I

using DynamicPolynomials
@polyvar x[1:5]
p = c'x - x' * Q * x / 2
using SumOfSquares
K = @set x[1] >= 0 && x[1] <= 1 &&
         x[2] >= 0 && x[2] <= 1 &&
         x[3] >= 0 && x[3] <= 1 &&
         x[4] >= 0 && x[4] <= 1 &&
         x[5] >= 0 && x[5] <= 1 &&
         10x[1] + 12x[2] + 11x[3] + 7x[4] + 4x[5] <= 40

Basic semialgebraic Set defined by no equality
11 inequalities
 x[1] ≥ 0
 1 - x[1] ≥ 0
 x[2] ≥ 0
 1 - x[2] ≥ 0
 x[3] ≥ 0
 1 - x[3] ≥ 0
 x[4] ≥ 0
 1 - x[4] ≥ 0
 x[5] ≥ 0
 1 - x[5] ≥ 0
 40 - 4*x[5] - 7*x[4] - 11*x[3] - 12*x[2] - 10*x[1] ≥ 0


We will now see how to find the optimal solution using Sum of Squares Programming.
We first need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [2]:
import Clarabel
solver = Clarabel.Optimizer

Clarabel.MOIwrapper.Optimizer

A Sum-of-Squares certificate that $p \ge \alpha$ over the domain `S`, ensures that $\alpha$ is a lower bound to the polynomial optimization problem.
The following function searches for the largest lower bound and finds zero using the `d`th level of the hierarchy`.

In [3]:
function solve(d)
    model = SOSModel(solver)
    @variable(model, α)
    @objective(model, Max, α)
    @constraint(model, c, p >= α, domain = K, maxdegree = d)
    optimize!(model)
    println(solution_summary(model))
    return model
end

solve (generic function with 1 method)

The first level of the hierarchy does not give any lower bound:

In [4]:
model2 = solve(2)
nothing # hide

-------------------------------------------------------------
           Clarabel.jl v0.9.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 33
  constraints   = 53
  nnz(P)        = 0
  nnz(A)        = 75
  cones (total) = 3
    : Zero        = 1,  numel = 21
    : Nonnegative = 1,  numel = 11
    : PSDTriangle = 1,  numel = 21

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

it

Indeed, as the constraints have degree 1 and their multipliers are SOS
so they have an even degree, with `maxdegree` 2 we can only use degree 0
multipliers hence constants. The terms of maximal degree in resulting
sum will therefore only be in `-x' * Q * x/2` hence it is not SOS whatever
is the value of the multipliers. Let's try with `maxdegree` 3 so that the
multipliers can be quadratic.
This second level is now feasible and gives a lower bound of `-22`.

In [5]:
model3 = solve(3)
nothing # hide

-------------------------------------------------------------
           Clarabel.jl v0.9.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 253
  constraints   = 308
  nnz(P)        = 0
  nnz(A)        = 715
  cones (total) = 13
    : Zero        = 1,  numel = 56
    : PSDTriangle = 12,  numel = (21,21,21,21,...,21)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost 

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*